In [1]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 43.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.
   ━━━━━━━━

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:

import warnings
warnings.filterwarnings("ignore")
import random
import numpy as np
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model
import evaluate

2025-12-03 11:09:48.301799: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764760188.493270      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764760188.550251      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [4]:
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
OUTPUT_DIR = "./boolq_lora_tinyllama_adaptive"
BATCH_SIZE = 16         
EVAL_BATCH = 32
NUM_EPOCHS = 3
LR = 1e-4
SEED = 42
MAX_LENGTH = 256
LORA_R = 8
LORA_ALPHA = 32
LORA_DROPOUT = 0.05
TARGET_MODULES =["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
ADAPTIVE_TOTAL_RANK = 672
ADAPTIVE_TAU = 1.0

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)

In [6]:
!pip install -U fsspec aiohttp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.5.0
    Uninstalling fsspec-2024.5.0:
      Successfully uninstalled fsspec-2024.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 2.20.0 requires fsspec[http]<=2024.5.0,>=2023.1.0, but you have fsspec 2025.10.0 which is incompatible.
s3fs 2025.3.0 requires fsspec==2025.3.0.*, but you have fsspec 2025.10.0 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.4 which is incompatible.
gc

In [7]:
from datasets import load_dataset
import time

def load_boolq_with_retry(max_retries=None, wait=2):
    attempt = 0

    while True:
        try:
            ds = load_dataset("boolq")
            print(f"✓ Dataset 'boolq' loaded successfully after {attempt} attempts.")
            return ds

        except Exception as e:
            attempt += 1
            wait_time = wait * min(5, attempt)  # exponential backoff with cap

            print(f"[Attempt {attempt}] Failed to load 'boolq': {e}")
            print(f"Retrying in {wait_time} seconds...\n")

            # Stop if max retries exceeded
            if max_retries is not None and attempt >= max_retries:
                print("✗ Maximum retries reached. Raising error.")
                raise e

            time.sleep(wait_time)
dataset = load_boolq_with_retry(max_retries=10)

[Attempt 1] Failed to load 'boolq': 429 Client Error: Too Many Requests for url: https://us.gcp.cdn.hf.co/xet-bridge-us/621ffdd236468d709f181d98/e5b5227060aa3f164d8ea38d88d1cebad9efc657e3d20236877a128008a80a4e?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27train-00000-of-00001.parquet%3B+filename%3D%22train-00000-of-00001.parquet%22%3B&Expires=1764763816&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzY0NzYzODE2fX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjIxZmZkZDIzNjQ2OGQ3MDlmMTgxZDk4L2U1YjUyMjcwNjBhYTNmMTY0ZDhlYTM4ZDg4ZDFjZWJhZDllZmM2NTdlM2QyMDIzNjg3N2ExMjgwMDhhODBhNGVcXD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=Zr-cSixX69g6DJ-EKgmCLmCS0q6xcEowNj8u7yYtzMda0Y2B8QAI1uzp18HqN5lfj9vEiI-gudaTsxAqmqImlR4hcPeXjMa~f8fNYqQvXziMjB52hD-7EK4PNKo~aTgZpbE~4EekZ6r7ZKtzAzvwRzucjnOGOStiblgyWhvr~lu1-Ttk-bcHQ4Tu13LNA7E0Yy78qyXP0zP4ITeXITIKuhdrL0d-tq~ifAuZCdLP~-BHrZ8mTPjWiYqCrAuOrVGBwmlJHtt8r9FxckZhWChivTbaQu

Generating train split:   0%|          | 0/9427 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3270 [00:00<?, ? examples/s]

✓ Dataset 'boolq' loaded successfully after 1 attempts.


In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 3270
    })
})

In [9]:

train_ds = dataset["train"]
val_ds = dataset["validation"]
print("Train size:", len(train_ds))
print("Validation (test) size:", len(val_ds))

Train size: 9427
Validation (test) size: 3270


In [10]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [11]:
def clean_text(s):
    if s is None:
        return ""
    return " ".join(str(s).strip().split())


def preprocess(batch):
    questions = [clean_text(q) for q in batch["question"]]
    passages  = [clean_text(p) for p in batch["passage"]]

    reformatted_questions = [
        f"Question: {q}"
        for q in questions
    ]

    reformatted_passages = [
        f"Passage: {p}"
        for p in passages
    ]

    toks = tokenizer(
        reformatted_questions,
        reformatted_passages,
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length",  # you can change to False + DataCollator if desired
    )

    # BoolQ labels: answer=True → entailment (1), answer=False → non-entailment (0)
    toks["labels"] = [1 if ans else 0 for ans in batch["answer"]]

    return toks


print("Preprocessing datasets...")
train_ds = train_ds.map(preprocess, batched=True)
val_ds = val_ds.map(preprocess, batched=True)

Preprocessing datasets...


Map:   0%|          | 0/9427 [00:00<?, ? examples/s]

Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

cols = ["input_ids", "attention_mask", "labels"]
train_ds.set_format(type="torch", columns=cols)
val_ds.set_format(type="torch", columns=cols)

train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=data_collator)
val_dataloader = DataLoader(val_ds, batch_size=EVAL_BATCH, shuffle=False, collate_fn=data_collator)

In [13]:
print("Model started Loading")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
    device_map="auto",
    torch_dtype=torch.bfloat16, 
    trust_remote_code=True
)

`torch_dtype` is deprecated! Use `dtype` instead!


Model started Loading


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
print("Model Loaded")

Model Loaded


In [15]:
model.config.pad_token_id = tokenizer.pad_token_id

In [16]:
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="SEQ_CLS",
)
model = get_peft_model(model, lora_config)

In [17]:
try:
    model.print_trainable_parameters()
except Exception:
    trainable, total = 0, 0
    for _, p in model.named_parameters():
        total += p.numel()
        if p.requires_grad:
            trainable += p.numel()
    print(f"Trainable params: {trainable:,} / {total:,} ({100*trainable/total:.4f}%)")
    

# --- METRICS & CALLBACKS ---
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=preds, references=labels)
    return {"accuracy": acc["accuracy"]}


trainable params: 6,311,936 || all params: 1,040,828,416 || trainable%: 0.6064


In [18]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=200,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    seed=SEED,                      
    gradient_accumulation_steps=2,      
    fp16=False,                             
    bf16=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("\nStarting training on BoolQ...")
trainer.train()


Starting training on BoolQ...


Epoch,Training Loss,Validation Loss,Accuracy
1,1.443500,0.610996,0.663914
2,1.180900,0.519213,0.730459
3,0.619400,0.527109,0.746361


TrainOutput(global_step=885, training_loss=1.0017832869190282, metrics={'train_runtime': 17971.3987, 'train_samples_per_second': 1.574, 'train_steps_per_second': 0.049, 'total_flos': 4.236632803875226e+16, 'train_loss': 1.0017832869190282, 'epoch': 3.0})

In [19]:
print("\nEvaluating final model...")
eval_result = trainer.evaluate(eval_dataset=val_ds)
test_accuracy = eval_result.get("eval_accuracy") or eval_result.get("accuracy")

print("\n=== Final testing (validation) results ===")
for k, v in eval_result.items():
    print(f"{k}: {v}")

if test_accuracy is not None:
    print(f"\nTest (validation) Accuracy: {test_accuracy:.4f}")
else:
    print("\nAccuracy not found in eval results. Full eval_result shown above.")

model.save_pretrained(OUTPUT_DIR)
print(f"\nLoRA adapter + model saved to {OUTPUT_DIR}")


Evaluating final model...



=== Final testing (validation) results ===
eval_loss: 0.5271088480949402
eval_accuracy: 0.7463608562691131
eval_runtime: 796.8895
eval_samples_per_second: 4.103
eval_steps_per_second: 0.129
epoch: 3.0

Test (validation) Accuracy: 0.7664

LoRA adapter + model saved to ./boolq_lora_tinyllama_adaptive
